<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


#8

In [16]:
#문법에 맞는 문자열을 반환하는 함수

#문법, LR table 과 비슷함
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    #마지막
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    #마지막
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output=[]
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        #다음 문자와 상태
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [17]:
np.random.seed(42)

for _ in range(25):
    #25개의 문자열 생성
    print(generate_string(default_reber_grammar), end=" ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

In [20]:
#모든 가능한 알파벳들
POSSIBLE_CHARS = "BTXXTTVPXTVPXTTVPSE"

#잘못된 문자열 만들기
def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    #올바른 문자열에
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    #있을수는 있지만 올바른 문자열엔 없는 문자열을
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    #중간에 하나 추가함
    return good_string[:index] + bad_char + good_string[index + 1:]

In [19]:
#잘못된 문자열
for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBPVVETV BTBTSSSPXXVVETE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE BPBPVVEPT BTBPTVEETE BTBTSSXXTTVXETE BTBTSXTTVVETE BPBPVVTPE BTBTSXTTVVETE EPBPVPXVVEPE BPTTXSEPE BPBTXXSPXTVVEPE BTBTXSPTE BPTTSXXTVPXVVEPE PPBPVPSEPE 